In [23]:
import numpy as np
import pandas as pd

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error,mean_squared_error,r2_score

import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import json

In [24]:
df=pd.read_csv('medical_insurance.csv')
df

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520
...,...,...,...,...,...,...,...
1333,50,male,30.970,3,no,northwest,10600.54830
1334,18,female,31.920,0,no,northeast,2205.98080
1335,18,female,36.850,0,no,southeast,1629.83350
1336,21,female,25.800,0,no,southwest,2007.94500


In [25]:
df.isna().sum()

age         0
sex         0
bmi         0
children    0
smoker      0
region      0
charges     0
dtype: int64

In [26]:
df.describe()

,age,bmi,children,charges
count,1338.000000,1338.000000,1338.000000,1338.000000
mean,39.207025,30.663397,1.094918,13270.422265
std,14.049960,6.098187,1.205493,12110.011237
min,18.000000,15.960000,0.000000,1121.873900
25%,27.000000,26.296250,0.000000,4740.287150
50%,39.000000,30.400000,1.000000,9382.033000
75%,51.000000,34.693750,2.000000,16639.912515
max,64.000000,53.130000,5.000000,63770.428010


In [27]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1338 entries, 0 to 1337
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       1338 non-null   int64  
 1   sex       1338 non-null   object 
 2   bmi       1338 non-null   float64
 3   children  1338 non-null   int64  
 4   smoker    1338 non-null   object 
 5   region    1338 non-null   object 
 6   charges   1338 non-null   float64
dtypes: float64(2), int64(2), object(3)
memory usage: 73.3+ KB


In [28]:
df['sex'].value_counts().to_dict()

{'male': 676, 'female': 662}

In [29]:
df['sex'].replace({'male': 0, 'female': 1},inplace=True)

In [30]:
df['smoker'].value_counts().to_dict()

{'no': 1064, 'yes': 274}

In [31]:
df['smoker'].replace({'no': 1, 'yes': 0},inplace=True)

In [32]:
df=pd.get_dummies(df,columns=['region'])


In [33]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1338 entries, 0 to 1337
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   age               1338 non-null   int64  
 1   sex               1338 non-null   int64  
 2   bmi               1338 non-null   float64
 3   children          1338 non-null   int64  
 4   smoker            1338 non-null   int64  
 5   charges           1338 non-null   float64
 6   region_northeast  1338 non-null   uint8  
 7   region_northwest  1338 non-null   uint8  
 8   region_southeast  1338 non-null   uint8  
 9   region_southwest  1338 non-null   uint8  
dtypes: float64(2), int64(4), uint8(4)
memory usage: 68.1 KB


In [34]:
df.describe()

,age,sex,bmi,children,smoker,charges,region_northeast,region_northwest,region_southeast,region_southwest
count,1338.000000,1338.000000,1338.000000,1338.000000,1338.000000,1338.000000,1338.000000,1338.000000,1338.000000,1338.000000
mean,39.207025,0.494768,30.663397,1.094918,0.795217,13270.422265,0.242152,0.242900,0.272048,0.242900
std,14.049960,0.500160,6.098187,1.205493,0.403694,12110.011237,0.428546,0.428995,0.445181,0.428995
min,18.000000,0.000000,15.960000,0.000000,0.000000,1121.873900,0.000000,0.000000,0.000000,0.000000
25%,27.000000,0.000000,26.296250,0.000000,1.000000,4740.287150,0.000000,0.000000,0.000000,0.000000
50%,39.000000,0.000000,30.400000,1.000000,1.000000,9382.033000,0.000000,0.000000,0.000000,0.000000
75%,51.000000,1.000000,34.693750,2.000000,1.000000,16639.912515,0.000000,0.000000,1.000000,0.000000
max,64.000000,1.000000,53.130000,5.000000,1.000000,63770.428010,1.000000,1.000000,1.000000,1.000000


# model training

In [35]:
x=df.drop('charges',axis=1)
y=df['charges']
x

,age,sex,bmi,children,smoker,region_northeast,region_northwest,region_southeast,region_southwest
0,19,1,27.900,0,0,0,0,0,1
1,18,0,33.770,1,1,0,0,1,0
2,28,0,33.000,3,1,0,0,1,0
3,33,0,22.705,0,1,0,1,0,0
4,32,0,28.880,0,1,0,1,0,0
...,...,...,...,...,...,...,...,...,...
1333,50,0,30.970,3,1,0,1,0,0
1334,18,1,31.920,0,1,1,0,0,0
1335,18,1,36.850,0,1,0,0,1,0
1336,21,1,25.800,0,1,0,0,0,1


In [36]:
y

0       16884.92400
1        1725.55230
2        4449.46200
3       21984.47061
4        3866.85520
           ...     
1333    10600.54830
1334     2205.98080
1335     1629.83350
1336     2007.94500
1337    29141.36030
Name: charges, Length: 1338, dtype: float64

In [37]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=5,shuffle=True)

In [38]:
x_train.shape

(1070, 9)

In [39]:
y_train.shape

(1070,)

In [40]:
x_test.shape

(268, 9)

In [41]:
y_test.shape

(268,)

In [42]:
model=LinearRegression()
model.fit(x_train,y_train)

LinearRegression()

In [43]:
#Model evaluation
y_p=model.predict(x_train)
mse=mean_squared_error(y_train,y_p)
print("mse",mse)
rmse=np.sqrt(mse)
print("rmse",rmse)

r2_score=model.score(x_train,y_train)
print(r2_score)

mse 37266065.520491056
rmse 6104.593804708963
0.7430471855293475


In [44]:
#Model evaluation
y_P=model.predict(x_test)
mse=mean_squared_error(y_test,y_P)
print("mse",mse)
rmse=np.sqrt(mse)
print("rmse",rmse)

r2_score=model.score(x_test,y_test)
print(r2_score)

mse 33680468.38188878
rmse 5803.487605043089
0.7781096604223829


# Testing on single row

In [45]:
x.head(1).T

,0
age,19.0
sex,1.0
bmi,27.9
children,0.0
smoker,0.0
region_northeast,0.0
region_northwest,0.0
region_southeast,0.0
region_southwest,1.0


In [46]:
age=19.0
sex='male'
bmi=27.9
children=0.0
smoker='no'
region_northeast=0.0
region_northwest=0.0
region_southeast=0.0
region_southwest=1.0

In [47]:
test_array=np.zeros(x.shape[1])
test_array

array([0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [48]:
project_data={"sex":{'male': 0, 'female': 1},
             'smoker':{'no': 1, 'yes': 0},
             'columns':list(x.columns)}
project_data['smoker'][smoker]

1

In [49]:
column_names=x.columns
column_names
region='region_'+'northeast'
region
region_index=np.where(column_names==region)[0][0]
region_index

5

In [52]:
test_array[0]=age
test_array[1]=project_data['sex'][sex]
test_array[2]=bmi
test_array[3]=children
test_array[4]=project_data['smoker'][smoker]
test_array[region_index]=1
test_array


array([19. ,  0. , 27.9,  0. ,  1. ,  1. ,  0. ,  0. ,  0. ])

In [53]:
model.predict([test_array])

F:\anaconda\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(


array([2057.41761525])

In [55]:
with open ('linear_model.pkl','wb') as f:
    pickle.dump(model,f)

In [56]:
with open ('project_data.json','w') as f:
    json.dump(project_data,f)